In [2]:
# Scenario Generation with Copulas 
# 
# Hugo S. de Araujo
# Nov. 14th, 2022 | Mays Group | Cornell University
################################################################################

#=======================================================================
PROJECT SETUP
=======================================================================#
using Pkg
Pkg.activate("copulas");
Pkg.instantiate();
# Import "here" function. Wrapper to allow easy path concatenation.
include(joinpath(@__DIR__, "functions", "fct_here.jl"))

# Import all required packages. 
begin
    # using AWSS3
    using CSV
    using DataFrames
    using Dates
    using DelimitedFiles
    using Distributions
    using HDF5
    using JuliaFormatter
    using LaTeXStrings
    using LinearAlgebra
    using LinearSolve
    #using Measures
    using Random
    using RCall
    using Revise
    using Statistics
    using StatsBase
    #using StatsPlots
    using OhMyREPL
    using Plots
    #using PrettyTables
    using Tables
    using TSFrames
    using TimeZones
end

# Include functions 
#= functions_dirpath = joinpath(pwd(),"src", "functions");
function_paths = readdir(functions_dirpath, join=true);
function_index = occursin.(".jl", function_paths);
functions_only = function_paths[function_index];

for str in functions_only
    include(str)
end =#

include(here("src", "functions", "fct_bind_historical_forecast.jl"));
include(here("src", "functions", "fct_compute_hourly_average_actuals.jl"));
include(here("src", "functions", "fct_compute_landing_probability.jl"));
include(here("src", "functions", "fct_convert_hours_2018.jl"));
include(here("src", "functions", "fct_convert_ISO_standard.jl"));
include(here("src", "functions", "fct_convert_land_prob_to_data.jl"));
include(here("src", "functions", "fct_generate_probability_scenarios.jl"));
include(here("src", "functions", "fct_getplots.jl"));
include(here("src", "functions", "fct_plot_correlation_heatmap.jl"));
include(here("src", "functions", "fct_plot_historical_landing.jl"));
include(here("src", "functions", "fct_plot_historical_synthetic_autocorrelation.jl"));
include(here("src", "functions", "fct_plot_correlogram_landing_probability.jl"));
include(here("src", "functions", "fct_plot_scenarios_and_actual.jl"));
include(here("src", "functions", "fct_read_h5_file.jl"));
include(here("src", "functions", "fct_read_input_file.jl"));
include(here("src", "functions", "fct_transform_landing_probability.jl"));
include(here("src", "functions", "fct_write_percentiles.jl"));
#=======================================================================
READ INPUT FILE
=======================================================================#
input_file_path = here("src\\copulas.txt")

# XXX Needs to be updated to be a hardcoded instead of reading in a text file
data_type,
scenario_length,
number_of_scenarios,
scenario_hour,
scenario_day,
scenario_month,
scenario_year,
read_locally,
historical_load,
forecast_load,
historical_solar,
forecast_da_solar,
forecast_2da_solar,
historical_wind,
forecastd_da_wind,
forecast_2da_wind,
write_percentile = read_input_file(input_file_path);

#=======================================================================
READ INPUT DATA: ARPA-E PERFORM PROJECT H5 FILES
=======================================================================#
# Function that reads the .h5 file and binds the time index and the actuals/fore-
# cast values into a single dataframe.

# Load data
load_actuals_raw = read_h5_file(here("data", historical_load), "load");
load_forecast_raw = read_h5_file(here("data", "ercot_BA_load_forecast_day_ahead_2018.h5"), "load", false);

# Solar data
solar_actuals_raw = read_h5_file(here("data", "ercot_BA_solar_actuals_Existing_2018.h5"), "solar");
solar_forecast_dayahead_raw = read_h5_file(here("data", "ercot_BA_solar_forecast_day_ahead_existing_2018.h5"), "solar", false);
solar_forecast_2dayahead_raw = read_h5_file(here("data", "ercot_BA_solar_forecast_2_day_ahead_existing_2018.h5"), "solar", false);

# Wind data
wind_actuals_raw = read_h5_file(here("data", "ercot_BA_wind_actuals_Existing_2018.h5"), "wind");
wind_forecast_dayahead_raw = read_h5_file(here("data", "ercot_BA_wind_forecast_day_ahead_existing_2018.h5"), "wind", false);
wind_forecast_2dayahead_raw = read_h5_file(here("data", "ercot_BA_wind_forecast_2_day_ahead_existing_2018.h5"), "wind", false);

#=======================================================================
Compute the hourly average for the actuals data
=======================================================================#
# Load
aux = compute_hourly_average_actuals(load_actuals_raw);
load_actual_avg_raw = DataFrame();
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
load_actual_avg_raw[!, :time_index] = time_index;
load_actual_avg_raw[!, :avg_actual] = avg_actual;

# Solar
aux = compute_hourly_average_actuals(solar_actuals_raw);
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
solar_actual_avg_raw = DataFrame();
solar_actual_avg_raw[!, :time_index] = time_index;
solar_actual_avg_raw[!, :avg_actual] = avg_actual;

# Wind
aux = compute_hourly_average_actuals(wind_actuals_raw);
time_index = aux[:, :Index];
avg_actual = aux[:, :values_mean];
wind_actual_avg_raw = DataFrame();
wind_actual_avg_raw[!, :time_index] = time_index;
wind_actual_avg_raw[!, :avg_actual] = avg_actual;

  Activating project at `c:\Users\ks885\Documents\aa_research\Modeling\norta_scenarios\copulas\src\copulas`


In [3]:
# create a function to do this for all data
function upd_convert_hours_2018(data, is_actual = true)
    if is_actual
        x = copy(data);
        x[:,:time_index] = x[:,:time_index] .- Dates.Hour(6);
        return x;
    else
        df = copy(data);
        df[:,:forecast_time] = df[:,:forecast_time] .- Dates.Hour(6);
        df[:,:issue_time] = df[:,:issue_time] .- Dates.Hour(6);
        return df;
    end
end

upd_convert_hours_2018 (generic function with 2 methods)

In [4]:
# Load data
load_actuals = upd_convert_hours_2018(load_actuals_raw);
load_actual_avg = upd_convert_hours_2018(load_actual_avg_raw);
load_forecast = upd_convert_hours_2018(load_forecast_raw, false);

# Solar data
solar_actuals = upd_convert_hours_2018(solar_actuals_raw);
solar_actual_avg = upd_convert_hours_2018(solar_actual_avg_raw);
solar_forecast_dayahead = upd_convert_hours_2018(solar_forecast_dayahead_raw, false);
solar_forecast_2dayahead = upd_convert_hours_2018(solar_forecast_2dayahead_raw, false);

# Wind data
wind_actuals = upd_convert_hours_2018(wind_actuals_raw);
wind_actual_avg = upd_convert_hours_2018(wind_actual_avg_raw);
wind_forecast_dayahead = upd_convert_hours_2018(wind_forecast_dayahead_raw, false);
wind_forecast_2dayahead = upd_convert_hours_2018(wind_forecast_2dayahead_raw, false);


In [5]:
#=======================================================================
BIND HOURLY HISTORICAL DATA WITH FORECAST DATA
========================================================================#
#= The binding is made by ("forecast_time" = "time_index"). This causes the 
average actual value to be duplicated, which is desired, given the # of rows
in the load_forecast is double that of load_actual. To distinguish a 
one-day-ahead forecast from a two-day-ahead forecast, the column "ahead_factor"
is introduced. Bind the day-ahead and two-day-ahead forecasts for wind and solar
to get all the forecast data into one object as it is for load forecast =#
    load_data = bind_historical_forecast(true,
    load_actual_avg,
    load_forecast);

solar_data = bind_historical_forecast(false,
    solar_actual_avg,
    solar_forecast_dayahead,
    solar_forecast_2dayahead);

wind_data = bind_historical_forecast(false,
    wind_actual_avg,
    wind_forecast_dayahead,
    wind_forecast_2dayahead);

In [6]:
#=======================================================================
Landing probability
=======================================================================#
#= This section holds the calculation of the probability that the actual
value was equaled or superior than the forecast percentiles for a given
day. This is made possible by the estimation of an approximate CDF
computed on the forecast percentiles. Once estimated, this function is
used to find the "landing probability"; the prob. that the actual value
is equal or greater than a % percentage of the forecast percentile.
=#
#include(here("src", "functions", "fct_compute_landing_probability.jl"))
landing_probability_load = compute_landing_probability(load_data);
landing_probability_solar = compute_landing_probability(solar_data);
landing_probability_wind = compute_landing_probability(wind_data);

In [7]:
#=======================================================================
ADJUST LANDING PROBABILITY DATAFRAME
=======================================================================#
lp_load = transform_landing_probability(landing_probability_load);
lp_solar = transform_landing_probability(landing_probability_solar);
lp_wind = transform_landing_probability(landing_probability_wind);

In [8]:
#=======================================================================
Determine length of Decision Problem
=======================================================================#
x = copy(wind_data);
# Sort data by issue time
sort!(x, :issue_time);
# Group data by issue time and count occurences in every group
df = combine(groupby(x, [:issue_time]), DataFrames.nrow => :count);
# Filter data by count. Only keep groups with 48 entries
df_filtered = filter(:count => ==(48), df);
issue_times_interest = df_filtered[!, :issue_time];
# find all forecast times for these issue times of interest
subset_wind_data = filter(row -> row[:issue_time] in issue_times_interest, wind_data);
subset_forecast_times = subset_wind_data[!, :forecast_time];
unique_forecast_times = unique(subset_forecast_times);
decision_T = length(unique_forecast_times);

unique_issue_times = unique(subset_wind_data[!, :issue_time]);

In [9]:
x = copy(lp_wind)
allequal_set = Set(findall(allequal, eachcol(x)));
allequal_ind = sort(collect(allequal_set));
allindex_set = Set(collect(1:48));
alldifferent_ind = sort(collect(setdiff(allindex_set, allequal_set))); # Index for columns whose st. dev. isn't zero
x = x[:, alldifferent_ind];


# ==================================================================
# COLUMNS TO KEEP
# ==================================================================
#=Here, we care only about the columns in x::DataFrame whose elements 
are not all equal. If they are, the correlation is not defined b/c
the standard deviation will be zero for columns whose elements
are all the same =#
if ishermitian(cor(x))
    Σ_Z = LinearAlgebra.cholesky(cor(x));
else
    Σ_Z = factorize(cor(x));
end
M = Σ_Z.L;
dim_M = size(M, 1);

# ==================================================================
# CORRELATION MATRIX
# ==================================================================
#= Determine a lower-triangular, nonsingular factorization M of the 
    the correlation matrix for Z such that MM' = Sigma_Z. =#
if ishermitian(cor(x))
    Σ_Z = LinearAlgebra.cholesky(cor(x));
else
    Σ_Z = factorize(cor(x));
end
M = Σ_Z.L;
dim_M = size(M, 1);



In [10]:
scenario_length

48

In [11]:
#define the actual landing probabilities as a vector
left_half = lp_solar[:, 1:size(lp_load, 2) ÷ 2];
q_landing_probability = vec(left_half);


In [12]:
println(scenario_year)
println(scenario_month)
println(scenario_day)
println(scenario_hour)   

2018
8
29
14


In [13]:
scenario_hour = 

ErrorException: syntax: incomplete: premature end of input

In [14]:
start_date = DateTime(string(scenario_year) * "-" * string(scenario_month) * "-" * string(scenario_day) * "T" * string(scenario_hour));

start_index = findfirst(isequal(start_date), unique_forecast_times)


5775

In [15]:
# ==================================================================
# PROBABILITY GENERATION LOOP
# ==================================================================
#= In certain cases, as in solar power, not all columns will be 
useful. Some will be discarded to avoid problems in the factorization
of the correlation matrix. Here we check if the dimension n of the 
matrix M (n x n) is equal to the scenario length stipulated as 48.
If it is not, the vector W will have its length adjusted to n. 
The variable allequal_ind stores the indices of the columns that 
were discarded. After the scenarios Y are generated, Y column dimension
will be expanded and all-zero columns will be introduced in the 
location of the allequal_ind
=#
#Random.seed!(29031990)
Random.seed!(12345)
Y = Matrix{Float64}(undef, number_of_scenarios, scenario_length)

need_expansion = 0 # This is specially important for solar data with several columns whose st. dev. is zero
if dim_M != scenario_length
    original_scen_length = scenario_length
    upd_scenario_length = dim_M
    Y = Matrix{Float64}(undef, number_of_scenarios, upd_scenario_length)
    need_expansion = 1
else
    upd_scenario_length = scenario_length;
end



48

In [16]:
start_index

5775

In [17]:
for nscen in 1:number_of_scenarios
    # Set up normal distribution with mean 0 and sd equal to 1
    d = Normal(0,1);

    #Generate vector W = (W_1, ..., W_k) ~ iid standard normal
    W = rand(d, upd_scenario_length);

    # Create vector Z such that Z <- MW
    Z = M * W;

    #Compute the CDF of Z
    #cdf_Z = sort(cdf.(d, Z));
    cdf_Z = cdf.(d, Z);
    
    for k in 1:upd_scenario_length
        #Apply the inverse CDF for X_k
        # Y[nscen, k] = quantile(x[:, k], cdf_Z[k])
        Y[nscen, k] = quantile(cdf_Z, q_landing_probability[start_index]);  
        #= tells us the simulated quantile that we are at \
        from the simulation scenario probability distribution...
        =#
    end
end

In [18]:
Y

1000×48 Matrix{Float64}:
 0.0794337  0.0794337  0.0794337  …  0.0794337  0.0794337  0.0794337
 0.342395   0.342395   0.342395      0.342395   0.342395   0.342395
 0.514108   0.514108   0.514108      0.514108   0.514108   0.514108
 0.280002   0.280002   0.280002      0.280002   0.280002   0.280002
 0.228564   0.228564   0.228564      0.228564   0.228564   0.228564
 0.263347   0.263347   0.263347   …  0.263347   0.263347   0.263347
 0.307214   0.307214   0.307214      0.307214   0.307214   0.307214
 0.392521   0.392521   0.392521      0.392521   0.392521   0.392521
 0.407655   0.407655   0.407655      0.407655   0.407655   0.407655
 0.380207   0.380207   0.380207      0.380207   0.380207   0.380207
 ⋮                                ⋱  ⋮                     
 0.337744   0.337744   0.337744      0.337744   0.337744   0.337744
 0.402702   0.402702   0.402702      0.402702   0.402702   0.402702
 0.478301   0.478301   0.478301      0.478301   0.478301   0.478301
 0.309806   0.309806   0.30980

Ok, the way I am doing this leads to the same probabilities every scenario... maybe I do need more work to understand emprical inverse cdf

In [19]:
# Set up normal distribution with mean 0 and sd equal to 1
d = Normal(0,1);

#Generate vector W = (W_1, ..., W_k) ~ iid standard normal
W = rand(d, upd_scenario_length);

# Create vector Z such that Z <- MW
Z = M * W;

#Compute the CDF of Z
#cdf_Z = sort(cdf.(d, Z));
cdf_Z = cdf.(d, Z);
print(cdf_Z)

[0.6748004358200773, 0.5969316924890833, 0.8745819583870414, 0.8769477455869276, 0.4694618534563613, 0.5828936020598221, 0.8286135813825537, 0.46239507314370715, 0.7608329596276878, 0.47537532409556404, 0.2867910099780618, 0.093332174608615, 0.1309993400221584, 0.09334281307268408, 0.46096788870721994, 0.18291436221988197, 0.23650285598916496, 0.625060038953213, 0.40664924639034605, 0.06921740616414815, 0.018194759811965236, 0.08650017774297397, 0.18448072570550295, 0.06590672884508846, 0.303027977831975, 0.19307859815738412, 0.44057102328982767, 0.16358861957163406, 0.10392848632535799, 0.1147071481337519, 0.17902991736402868, 0.08527916157772619, 0.08963466518215298, 0.016118462980960594, 0.0897352794632893, 0.17435494368729143, 0.44796204862330247, 0.6576778476018162, 0.6775980570366191, 0.40718276164921435, 0.5389914504034322, 0.45648409407930857, 0.8889062370555455, 0.6397709441437422, 0.2629740073649409, 0.42816626128117036, 0.38422856721425186, 0.2577728796605652]

In [20]:
cdf_Z[1]

0.6748004358200773

In [21]:
for nscen in 1:number_of_scenarios
    # Set up normal distribution with mean 0 and sd equal to 1
    d = Normal(0,1);

    #Generate vector W = (W_1, ..., W_k) ~ iid standard normal
    W = rand(d, upd_scenario_length);

    # Create vector Z such that Z <- MW
    Z = M * W;

    #Compute the CDF of Z
    #cdf_Z = sort(cdf.(d, Z));
    cdf_Z = cdf.(d, Z);
    
    for k in 1:upd_scenario_length
        #Apply the inverse CDF for X_k
        # Y[nscen, k] = quantile(x[:, k], cdf_Z[k])
        # Y[nscen, k] = quantile(cdf_Z, q_landing_probability[start_index]);  
        Y[nscen, k] = quantile(q_landing_probability[1:start_index], cdf_Z[k])
        #= tells us the simulated quantile that we are at \
        from the simulation scenario probability distribution...
        =#
    end
end

Y

1000×48 Matrix{Float64}:
 1.0       1.0       1.0       1.0        …  1.0       1.0       0.555556
 0.242424  0.494949  0.0       0.434343      0.585859  1.0       0.606061
 1.0       1.0       1.0       1.0           1.0       1.0       0.750032
 1.0       1.0       1.0       1.0           0.89899   0.161616  1.0
 1.0       1.0       0.941547  0.59596       1.0       1.0       1.0
 1.0       1.0       0.494949  1.0        …  1.0       0.828283  1.0
 1.0       0.69697   0.909091  0.0           0.262626  0.232323  0.434343
 1.0       1.0       1.0       1.0           0.818182  0.838384  1.0
 0.272727  1.0       0.636364  0.10101       0.929293  1.0       0.222222
 1.0       1.0       1.0       1.0           1.0       1.0       0.686869
 ⋮                                        ⋱  ⋮                   
 0.606061  0.69697   1.0       0.393939      1.0       1.0       1.0
 0.535354  1.0       0.444444  0.0909091     0.636364  0.585859  0.555556
 1.0       1.0       0.676768  1.0           1

In [22]:
# Set up normal distribution with mean 0 and sd equal to 1
d = Normal(0,1);

#Generate vector W = (W_1, ..., W_k) ~ iid standard normal
W = rand(d, upd_scenario_length);

# Create vector Z such that Z <- MW
Z = M * W;

#Compute the CDF of Z
#cdf_Z = sort(cdf.(d, Z));
cdf_Z = cdf.(d, Z);

In [23]:
sort(cdf_Z)

48-element Vector{Float64}:
 0.009359347806417721
 0.01141310984175262
 0.014104029366137368
 0.019795250076194097
 0.024944973917460866
 0.02778812076918316
 0.05576288654630733
 0.07532450113693266
 0.09489087025309362
 0.09597483365020541
 ⋮
 0.6080172995751342
 0.6093142347758218
 0.6211955865136524
 0.646527384811223
 0.6642395278068141
 0.7509829664024829
 0.7761808581188159
 0.7949557689730458
 0.8645293422980851

In [24]:
quantile(cdf_Z, 0.5)

0.38036319508116084

We should have a different cdf_Z every time though because our Z is a random process.

In [25]:
# Set up normal distribution with mean 0 and sd equal to 1
d = Normal(0,1);

#Generate vector W = (W_1, ..., W_k) ~ iid standard normal
W = rand(d, upd_scenario_length);

# Create vector Z such that Z <- MW
Z = M * W;

#Compute the CDF of Z
#cdf_Z = sort(cdf.(d, Z));
cdf_Z = cdf.(d, Z);

In [26]:
sort(cdf_Z)

48-element Vector{Float64}:
 0.0006464586292966945
 0.0009111056081107798
 0.0018514343774372762
 0.005817585254652345
 0.023309494938876175
 0.029367030693725604
 0.0324864353585237
 0.04948114634109986
 0.11417916024761457
 0.11590626041498091
 ⋮
 0.6031388875832259
 0.6111189687926744
 0.6502578259086018
 0.6591511811277921
 0.6762421357564288
 0.6858145000488965
 0.733299162043397
 0.7652974468304492
 0.8383487254584268

And we do.
??? So why do we get the same quantile estimate for each scenario when we run the loop?

In [27]:
q_landing_probability[start_index]

0.35353535353535354

In [28]:
cdf_Z = [];

for nscen in 1:number_of_scenarios
    # Set up normal distribution with mean 0 and sd equal to 1
    d = Normal(0,1);

    #Generate vector W = (W_1, ..., W_k) ~ iid standard normal
    W = rand(d, upd_scenario_length);

    # Create vector Z such that Z <- MW
    Z = M * W;

    #Compute the CDF of Z
    #cdf_Z = sort(cdf.(d, Z));
    cdf_Z = cdf.(d, Z);
    
    for k in 1:upd_scenario_length
        #Apply the inverse CDF for X_k
        # Y[nscen, k] = quantile(x[:, k], cdf_Z[k])
        Y[nscen, k] = quantile(cdf_Z, q_landing_probability[start_index]);  
        #= tells us the simulated quantile that we are at \
        from the simulation scenario probability distribution...
        =#
    end
end

sort(cdf_Z)

48-element Vector{Float64}:
 0.009083853225206271
 0.02336085994577465
 0.029891854122093366
 0.033394369921069686
 0.04441874539518969
 0.05259749439515141
 0.05410010868538209
 0.057563644715424606
 0.057837659501386376
 0.09797119514901093
 ⋮
 0.7545935250613748
 0.7610998325902081
 0.7805174067433231
 0.7888553881409697
 0.7920409316503734
 0.8264582353777687
 0.8512328669622009
 0.8880945198723672
 0.8898200442270565

It's not in the cdf

In [29]:
cdf_Z = [];

for nscen in 1:number_of_scenarios
    # Set up normal distribution with mean 0 and sd equal to 1
    d = Normal(0,1);

    #Generate vector W = (W_1, ..., W_k) ~ iid standard normal
    W = rand(d, upd_scenario_length);

    # Create vector Z such that Z <- MW
    Z = M * W;

    #Compute the CDF of Z
    #cdf_Z = sort(cdf.(d, Z));
    cdf_Z = cdf.(d, Z);
    
    for k in 1:upd_scenario_length
        #Apply the inverse CDF for X_k
        # Y[nscen, k] = quantile(x[:, k], cdf_Z[k])
        Y[nscen, k] = quantile(cdf_Z, q_landing_probability[start_index]);  
        #= tells us the simulated quantile that we are at \
        from the simulation scenario probability distribution...
        =#
    end
end

It's in the lookahead loop because for each lookahead we are using the same cdf. 

I don't see what to do then.

In [30]:
quantile(cdf_Z)

5-element Vector{Float64}:
 0.011830702825656346
 0.14070050631657416
 0.286617669797955
 0.641813632799082
 0.9200217188282381

* why is this a 5 element vector? 
* I assume it is at least the quantiles, maybe the 1/5 quantiles?

In [56]:
# quantile.(cdf_Z, q_landing_probability[start_index])
quantile(cdf_Z, rand())

0.04244556112461114

In [58]:
for nscen in 1:number_of_scenarios
    # Set up normal distribution with mean 0 and sd equal to 1
    d = Normal(0,1);

    #Generate vector W = (W_1, ..., W_k) ~ iid standard normal
    W = rand(d, upd_scenario_length);

    # Create vector Z such that Z <- MW
    Z = M * W;

    #Compute the CDF of Z
    #cdf_Z = sort(cdf.(d, Z));
    cdf_Z = cdf.(d, Z);
    
    for k in 1:upd_scenario_length
        #Apply the inverse CDF for X_k
        # Y[nscen, k] = quantile(x[:, k], cdf_Z[k])
        Y[nscen, k] = quantile(cdf_Z, rand());  
        #= tells us the simulated quantile that we are at \
        from the simulation scenario probability distribution...
        =#
    end
end

Y

1000×48 Matrix{Float64}:
 0.0503401  0.0351006  0.477001   …  0.384524  0.0892249  0.457553
 0.666941   0.888415   0.937561      0.950957  0.843607   0.790617
 0.169702   0.370392   0.0678211     0.739394  0.802805   0.714662
 0.901052   0.855679   0.566976      0.331315  0.400218   0.710087
 0.797804   0.648861   0.562804      0.430488  0.937583   0.196515
 0.712084   0.673599   0.65135    …  0.964912  0.89849    0.63247
 0.531994   0.356375   0.856329      0.761692  0.238182   0.183674
 0.735221   0.692423   0.739426      0.816293  0.229197   0.816081
 0.176761   0.19481    0.272236      0.549988  0.67602    0.183151
 0.397918   0.724999   0.416465      0.868101  0.817493   0.0075125
 ⋮                                ⋱  ⋮                    
 0.866206   0.211128   0.886371      0.688613  0.10672    0.00101522
 0.979847   0.998674   0.463339      0.180182  0.987108   0.00193644
 0.691154   0.521563   0.847635      0.49733   0.560506   0.372405
 0.990309   0.556445   0.864403      0.98

- I'm not sure if this is correct.
- also, if this were load, the ramping as a result could be catastrophic for the optimization model.
- there is no conditioning on the current time period too. so this is wrong